# ANN - Iris
- TensorFlow & Keras 
-3 class (Multi-class Classification)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Dataset

In [ ]:
df = sns.load_dataset('iris')
# df.head()
df.sample(5, random_state=1)

In [ ]:
df.isnull().sum()

In [ ]:
sns.scatterplot(x='petal_length', y='petal_width', data=df, hue='species', style='species', 
                alpha=0.85, edgecolor='w', s=100)
plt.show()

In [ ]:
class_names = np.unique(df.species)
class_names

## X and y

In [ ]:
X = df.drop('species', axis=1)
# X.head(6)
X.sample(6, random_state=1)

In [ ]:
X[:5]

In [ ]:
y = df.species

In [ ]:
y[:5]
# y[45:105]

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_sc = sc.fit_transform(X)
X_sc[:4]

## Train-test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.25, random_state=1) #

X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape 

In [ ]:
y_train[:5]

In [ ]:
y_test[:5]

## One-Hot Encoding

In [ ]:
from tensorflow.keras.utils import to_categorical

def one_hot_encode(arr):
    categories, ids = np.unique(arr, return_inverse=True)
    print(categories)
    return to_categorical(ids, len(categories))

y_train_1h = one_hot_encode(y_train).astype('int')
y_test_1h = one_hot_encode(y_test).astype('int')

In [ ]:
y_train_1h[:4]

In [ ]:
y_test_1h[:4]

In [ ]:
y_train_1h.shape, y_test_1h.shape 

In [ ]:
X_train[:4]

In [ ]:
y_train[:4]

In [ ]:
y_train_1h[:4]

## Model
- Create a Model
- Compile
- Train

### Create


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense 

model = Sequential()

hidden_node = 64
model.add(Dense(hidden_node, activation='relu', input_shape=(4,))) 
# model.add(Dense(128, activation='relu))

model.add(Dense(3, activation='softmax'))     # 3 class ขึ้นไป (Multi-class classification)
# model.add(Dense(1, activation='sigmoid'))   # 2 class (Binary classification)

In [ ]:
model.summary()

In [ ]:
model.output_shape

In [ ]:
# model.get_config()

### Compile & Train

In [ ]:
model.compile(loss='categorical_crossentropy',  # รองรับ Classification 3 class ขึ้นไป
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# categorical_crossentropy ใช้ y train แบบ One-hot Encoding

import time

start = time.time()
history = model.fit(X_train, y_train_1h, epochs=80, batch_size=64, verbose=1, validation_split=0.25) 
end = time.time()

print(f"Time Taken {end - start:.3f} secs")

## Evaluate

In [ ]:
score = model.evaluate(X_test, y_test_1h, verbose=0)

print("Test loss:", score[0])
print(f"Test accuracy: {score[1]:.4f}")

In [ ]:
model.metrics_names

In [ ]:
y_pred = model.predict(X_test)

y_pred[:4]
y_pred[:4].round(3)

In [ ]:
y_pred_cat = np.argmax(y_pred, axis=1)
y_pred_cat[:4]

In [ ]:
y_test[:4]

In [ ]:
_, y_test_enc = np.unique(y_test, return_inverse=True)  # Label encoding
y_test_enc[:4]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test_enc, y_pred_cat, target_names=class_names))
print(confusion_matrix(y_test_enc, y_pred_cat))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test_enc, y_pred_cat)   ## <-- 
ConfusionMatrixDisplay(cm, display_labels=class_names).plot()
plt.title('Confusion Matrix')
plt.show()

## Loss and Accuracy Curves

In [ ]:
# 2 Columns
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)   # row column index
plt.plot(history.history['loss'], '--', c='b', lw=2, label='Trainning')
plt.plot(history.history['val_loss'], c='r', lw=3, label='Validation')
plt.title('Loss Curve')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)   # row column index
plt.plot(history.history['accuracy'], '--', c='b', lw=2, label='Trainning')
plt.plot(history.history['val_accuracy'], c='r', lw=3, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Accuracy Curve')
plt.show()

## Predict

In [ ]:
X_new = [[8.1, 4.0, 5.9, 2.1], #  1
         [6.8, 3.6, 3.5, 1.4]] #  2

# X_new = [[8.1, 4.0, 5.9, 2.1]]

X_new_sc = sc.transform(X_new)
X_new_sc

In [ ]:
y_pred_pr = model.predict(X_new_sc)
y_pred_pr.round(3)

In [ ]:
y_pred = np.argmax(y_pred_pr, axis=1)
y_pred

In [ ]:
print(class_names[y_pred])

## Decision Regions

In [ ]:
class Onehot2Int(object):
    def __init__(self, model):
        self.model = model

    def predict(self, X):
        y_pred = self.model.predict(X)
        return np.argmax(y_pred, axis=1)
    
model_no_ohe = Onehot2Int(model)    

In [ ]:
from mlxtend.plotting import plot_decision_regions

value = 0
width = 5.5

ax = plot_decision_regions(X_test, np.array(y_test_enc),

                    clf=model_no_ohe, feature_index=[2, 3],  
                    filler_feature_values={0: value, 1: value},
                    filler_feature_ranges={0: width, 1: width},
                    legend=2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, class_names, framealpha=0.5)

plt.title('Iris (Hidden={})'.format(hidden_node))
# plt.xticks([])
plt.yticks([])
plt.xlabel('petal_length')
plt.ylabel('petal_width')

plt.scatter(X_new_sc[:,2][y_pred==0], X_new_sc[:,3][y_pred==0], marker='s', s=180, c='b')
plt.scatter(X_new_sc[:,2][y_pred==1], X_new_sc[:,3][y_pred==1], marker='^', s=230, c='#7d4128') # 
plt.scatter(X_new_sc[:,2][y_pred==2], X_new_sc[:,3][y_pred==2], marker='o', s=150, c='b')
plt.show()

## แบบฝึกหัด

- epochs 100 (เท่าเดิม) batch_size 64 (ใหญ่ขึ้น) ความแม่นยำจะได้น้อยกว่า เนื่องจาก batch_size ขนาดใหญ่ขึ้นทำให้จำนวนรอบย่อย (Iterations) ลดลง ถึงแม้ epochs เท่าเดิม เส้นกราฟของพล็อต Loss ยังไม่นิ่ง ดูเหมือนว่าจะมีแนวโน้มลงไปได้อีก
- epochs 100 (เท่าเดิม) batch_size (เล็กลง) ความแม่นยำจะได้ประมาณสูง แต่ละ epochs มีการวนรอบย่อย (Iterations) มากขึ้น เนื่องจาก batch_size ขนาดเล็กลง เมื่อ ดูกราฟของพล็อต Loss และ Accuracy พบว่า ประมาณ epoch 40 Loss เริ่มนิ่งแล้ว และ Accuracy ได้ค่าสูงสุด จากนั้น Train epoch ต่อไป ทำให้ Accuracy ตกลง ลักษณะ นี้ epochs สูงเกินไป ใช้เพียงแค่ 40-50 ก็เหมาะสมแล้ว (Train มากแล้วแย่ลง)
- epochs 10, batch_size 32
- เปลี่ยน Dense เป็น 8 หรือ 16
- ลองเพิ่มจำนวนชั้น Layer แล้วดูพล็อตขอบเขตการจำแนกของ Model